# Getting Started with Azure OpenAI: Reproducible Outputs

By default, responses from the Azure OpenAI chat completion model are nondeterministic - asking the same question multiple times can yield different answers.

If you want to obtain reproducible output for the same question, one solution is to set the temperature to 0. However, setting temperature to 0 limits creativity in responses.

A better solution for consistent output is to use the optional `seed` parameter.

The seed parameter accepts an integer value. When provided, the model makes a "best effort" to return the same result for the same parameters and same seed value.

**Note:** Determinism isn't 100% guaranteed. Even in cases where the seed value and all other parameters are the same across API calls, it's not uncommon to still observe a degree of variability in responses.

## IMPORTANT NOTE

As of September 2025, the `seed` parameter is NOT available in OpenAI's Responses API. It is only supported in the older Chat Completions API.

Reference:
  - Ticket: https://github.com/openai/openai-node/issues/1613

This means you CAN use the `seed` parameter with Azure OpenAI models, but you are restricted to using the Chat Completions API.

Related Discussions:
  - https://community.openai.com/t/support-for-seed-parameter-in-the-responses-api/1230489/2
  - https://community.openai.com/t/seed-parameter-use-in-4-1-and-reasoning-models-too/1298703

Because of this limitation, this example uses the Chat Completions API instead of the Responses API.

## Prerequisites

1. Make sure that python3 is installed on your system.
2. Create and Activate a Virtual Environment:
   - `python3 -m venv venv`
   - `source venv/bin/activate`
3. The required libraries are listed in the requirements.txt file. Use the following command to install them:
   - `pip3 install -r requirements.txt`
4. Create a `.env` file in the same directory as this script and add the following variables:
   - `AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>`
   - `AZURE_OPENAI_MODEL=<your_azure_openai_model>`
   - `AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>`
   - `AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>`

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [82]:
! pip install -r requirements.txt

## Import Required Modules

In [83]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.

## Load Environment Variables

The `load_dotenv()` function reads the .env file and loads the variables as environment variables, making them accessible via `os.environ` or `os.getenv()`.

**Note:** 
- `os.environ[]` raises an exception if the variable is not found
- `os.getenv()` does not raise an exception, but returns None

In [84]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create Azure OpenAI Client

The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API. It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [85]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Define the Question

Let's define a question to demonstrate the difference between deterministic and non-deterministic responses.

In [86]:
QUESTION = "Tell me a short urban legend in 3 lines"
print("=" * 80)
print(f"Question to ask the model: {QUESTION}")
print("=" * 80)

Question to ask the model: Tell me a short urban legend in 3 lines


## Other Parameters

In [87]:

SYSTEM_PROMPT = "You are a great storyteller."
TEMPERATURE = 0.9
MAX_TOKENS = 100

## Demonstrate Non-deterministic Responses (Without Seed)

First, let's generate three responses to the same question without using the seed parameter to show the natural variability in responses.

In [88]:
print("=" * 80)
print("Generating 3 responses to the same question without using seed parameter...")
print("=" * 80)

for i in range(3):
    print(f'Story Version {i + 1}')
    print('---')
    
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            # Note: No seed parameter specified
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": QUESTION}
            ]
        )
        
        print(response.choices[0].message.content)
        print("---\n")
        
    except Exception as e:
        print(f"Error getting response: {e}")
        continue


Generating 3 responses to the same question without using seed parameter...
Story Version 1
---
In a bustling city, a woman receives a text from her best friend saying she's in trouble and needs help. Rushing to the location, she finds her friend's phone lying on the ground, but when she picks it up, a chilling voice whispers from the darkness, "You shouldn't have come alone." The next day, her friend's family receives a message saying she was never seen again.
---

Story Version 2
---
In a bustling city, a mysterious woman in red appears at midnight, whispering secrets of the past. Those who listen are said to vanish without a trace by dawn, leaving only a single red rose behind. Locals warn that if you find the rose, it's already too late.
---

Story Version 3
---
In a bustling city, a lonely woman would often hear a faint whispering her name in the dark alleys. One night, driven by curiosity, she followed the sound only to find a mirror reflecting her own terrified face, but behind 

## Demonstrate Reproducible Responses (With Seed = 42)

Now let's generate multiple responses to the same question using the same seed value to show how responses become more consistent and reproducible.

In [89]:
print("=" * 80)
print("Generating 3 responses to the same question using seed=42...")
print("=" * 80)

for i in range(3):
    print(f'Story Version {i + 1}')
    print('---')
    
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            seed=42,  # Setting seed for reproducible outputs
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": QUESTION}
            ]
        )
        
        print(response.choices[0].message.content)
        print("---\n")
        
    except Exception as e:
        print(f"Error getting response: {e}")
        continue

Generating 3 responses to the same question using seed=42...
Story Version 1
---
In a bustling city, a woman always noticed a man in a fedora waiting at the same bus stop. One night, she struck up a conversation and discovered he had died decades ago, buried in the very spot where they stood. The next day, the bus stop was gone, replaced by a small park—his resting place.
---

Story Version 2
---
In a bustling city, a woman always noticed a man in a fedora waiting at the same bus stop. One night, she struck up a conversation and discovered he had died decades ago, buried in the very spot where he stood. The next morning, the bus stop was gone, replaced by a small park—his resting place.
---

Story Version 3
---
In a bustling city, a woman always noticed a man in a fedora waiting at the same bus stop. One night, she struck up a conversation and discovered he had died decades ago, buried in the very spot where they stood. The next day, the bus stop was gone, replaced by a small park—his 

## Demonstrate Reproducible Responses (With Seed = 20)

Let's try with a different seed value to show how different seeds produce different but consistent results.

In [90]:
print("=" * 80)
print("Generating 3 responses to the same question using seed=20...")
print("=" * 80)

for i in range(3):
    print(f'Story Version {i + 1}')
    print('---')
    
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            seed=20,  # Setting seed for reproducible outputs
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": QUESTION}
            ]
        )
        
        print(response.choices[0].message.content)
        print("---\n")
        
    except Exception as e:
        print(f"Error getting response: {e}")
        continue

Generating 3 responses to the same question using seed=20...
Story Version 1
---
Late at night, a woman in a white dress appears in the rearview mirror of cars on an empty highway. If you stop to help her, she vanishes, leaving only a chilling coldness behind. The next morning, drivers find their gas tanks mysteriously empty, as if she drained their fuel to escape.
---

Story Version 2
---
Late at night, a woman in white stands on the side of a deserted road, hitchhiking. When drivers stop to help, she vanishes from the backseat, leaving only a cold chill behind. The next morning, they find her missing photo on a nearby missing person’s poster—she disappeared decades ago.
---

Story Version 3
---
Late at night, a woman in white stands on the side of a deserted road, hitchhiking. When drivers stop to help, she vanishes from the backseat, leaving only a cold chill behind. The next morning, they find her missing photo on a nearby missing person’s poster—she disappeared decades ago.
---



## Key Observations

1. **Without seed**: Each response will likely be different, showing the natural variability of the model.

2. **With seed=42**: The three responses should be very similar or identical, demonstrating reproducibility.

3. **With seed=20**: The responses will be consistent among themselves but different from the seed=42 responses.